# How to inspect model fit results

In [1]:
from rlssm.utility.utils import load_model_results

# load non-hierarchical DDM fit:
model_fit_ddm = load_model_results('/Users/laurafontanesi/git/rlssm/docs/notebooks/DDM.pkl')

# load non-hierarchical LBA fit:
model_fit_lba = load_model_results('/Users/laurafontanesi/git/rlssm/docs/notebooks/LBA_2A.pkl')

# load hierarchical RL fit:
model_fit_rl = load_model_results('/Users/laurafontanesi/git/rlssm/docs/notebooks/hierRL_2A.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/laurafontanesi/git/rlssm/docs/notebooks/DDM.pkl'

## Posteriors

The posterior samples are stored in `samples`:

In [ ]:
model_fit_ddm.samples

In [ ]:
model_fit_rl.samples.describe()

You can simply plot the model's posteriors using `plot_posteriors`: 

In [ ]:
model_fit_ddm.plot_posteriors();

By default, 95% HDIs are shown, but you can also choose to have the posteriors without intervals or BCIs, and change the alpha level:

In [ ]:
model_fit_rl.plot_posteriors(show_intervals='BCI', alpha_intervals=.01);

### Trial-level

Depending on the model specification, you can also extract certain trial-level parameters as numpy ordered dictionaries of n_samples X n_trials shape:

In [ ]:
model_fit_ddm.trial_samples['drift_t'].shape

In [ ]:
model_fit_ddm.trial_samples.keys()

In [ ]:
model_fit_lba.trial_samples.keys() # for the LBA

In the case of a RL model fit on choices alone, you can extract the log probability of accuracy=1 for each trial:

In [ ]:
model_fit_rl.trial_samples.keys()

In [ ]:
model_fit_rl.trial_samples['log_p_t'].shape

## Posterior predictives

With `get_posterior_predictives_df` you get posterior predictives as pandas DataFrames of `n_posterior_predictives` X `n_trials` shape:

In [ ]:
pp = model_fit_rl.get_posterior_predictives_df(n_posterior_predictives=1000)
pp

For the DDM, you have additional parameters to tweak the DDM simulations, and you get a DataFrame with a hierarchical column index,  for RTs and for accuracy:

In [ ]:
pp = model_fit_ddm.get_posterior_predictives_df(n_posterior_predictives=100, dt=.001)
pp

You can also have posterior predictive summaries with `get_posterior_predictives_summary`.

Only mean accuracy for RL models fit on choices alone, and also mean RTs, skewness and quantiles for lower and upper boundaries for models fitted on RTs as well.

In [ ]:
model_fit_rl.get_posterior_predictives_summary()

In [ ]:
model_fit_ddm.get_posterior_predictives_summary()

You can also specify which quantiles you are interested in:

In [ ]:
model_fit_lba.get_posterior_predictives_summary(n_posterior_predictives=200, quantiles=[.1, .5, .9])

Finally, you can get summary for grouping variables (e.g., experimental conditions, trial blocks, etc.) in your data:

In [ ]:
model_fit_lba.get_grouped_posterior_predictives_summary(n_posterior_predictives=200,
                                                        grouping_vars=['block_label'],
                                                        quantiles=[.3, .5, .7])

## Plot posterior predictives

You can plot posterior predictives similarly, both **ungrouped** (across all trials) or **grouped** (across conditions, trial blocks, etc.plot_mean_posterior_predictives).

For RT models, you have both **mean plots**, and **quantile plots**:

In [ ]:
model_fit_ddm.plot_mean_posterior_predictives(n_posterior_predictives=200);

Quantile plots have 2 main visualization options, "shades" and "lines", and you can specify again which quantiles you want, which in tervals and alpha levels: 

In [ ]:
model_fit_lba.plot_quantiles_posterior_predictives(n_posterior_predictives=200);

In [ ]:
model_fit_lba.plot_quantiles_posterior_predictives(n_posterior_predictives=200,
                                                   kind='shades',
                                                   quantiles=[.1, .5, .9]);

In [ ]:
model_fit_lba.plot_quantiles_grouped_posterior_predictives(
    n_posterior_predictives=100,
    grouping_var='block_label',
    kind='shades',
    quantiles=[.1, .3, .5, .7, .9]);

In [ ]:
# Define new grouping variables:

import pandas as pd
import numpy as np

data = model_fit_rl.data_info['data']

# add a column to the data to group trials across learning blocks
data['block_bins'] = pd.cut(data.trial_block, 8, labels=np.arange(1, 9))

# add a column to define which choice pair is shown in that trial
data['choice_pair'] = 'AB'
data.loc[(data.cor_option == 3) & (data.inc_option == 1), 'choice_pair'] = 'AC'
data.loc[(data.cor_option == 4) & (data.inc_option == 2), 'choice_pair'] = 'BD'
data.loc[(data.cor_option == 4) & (data.inc_option == 3), 'choice_pair'] = 'CD'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(20,8))

model_fit_rl.plot_mean_grouped_posterior_predictives(grouping_vars=['block_bins'], n_posterior_predictives=500, ax=axes[0])

model_fit_rl.plot_mean_grouped_posterior_predictives(grouping_vars=['block_bins', 'choice_pair'], 
                                                     n_posterior_predictives=500, ax=axes[1])

sns.despine()